In [ ]:


import os
import nibabel as nib
import numpy as np
import torch
from torch.utils.data import Dataset
import shutil



class BraTSDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.patients = sorted(os.listdir(root_dir))

    def __len__(self):
        return len(self.patients)

    def __getitem__(self, idx):
        patient_id = self.patients[idx]
        patient_path = os.path.join(self.root_dir, patient_id)

        # Chargement des modalités séparément pour chaque patient
        flair = nib.load(os.path.join(patient_path, f"{patient_id}_flair.nii.gz")).get_fdata()
        t1 = nib.load(os.path.join(patient_path, f"{patient_id}_t1.nii.gz")).get_fdata()
        t1ce = nib.load(os.path.join(patient_path, f"{patient_id}_t1ce.nii.gz")).get_fdata()
        t2 = nib.load(os.path.join(patient_path, f"{patient_id}_t2.nii.gz")).get_fdata()

        # Chargement du masque (segmentation)
        mask = nib.load(os.path.join(patient_path, f"{patient_id}_seg.nii.gz")).get_fdata()

        # Conversion en tenseur PyTorch
        flair = torch.tensor(flair, dtype=torch.float32).unsqueeze(0)
        t1 = torch.tensor(t1, dtype=torch.float32).unsqueeze(0)
        t1ce = torch.tensor(t1ce, dtype=torch.float32).unsqueeze(0)
        t2 = torch.tensor(t2, dtype=torch.float32).unsqueeze(0)
        mask = torch.tensor(mask, dtype=torch.float32).unsqueeze(0)

        return flair, t1, t1ce, t2, mask


#  le chemin de mon dataset
root_dir = "BraTS2021_Augmented"

# Créatuin d'une instance du dataset
dataset = BraTSDataset(root_dir)

# Vérification du taille du dataset
print(f"Nombre total de patients dans le dataset : {len(dataset)}")

# Teste un exemple
flair, t1, t1ce, t2, mask = dataset[0]

# Affichage de la forme des données
print(f"Flair shape: {flair.shape}")
print(f"T1 shape: {t1.shape}")
print(f"T1ce shape: {t1ce.shape}")
print(f"T2 shape: {t2.shape}")
print(f"Mask shape: {mask.shape}")